<a href="https://colab.research.google.com/github/knobs-dials/wetsuite-dev/blob/main/notebooks/extras/extras_find_references.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# (only) in colab, run this first to install wetsuite from (the most recent) source.   For your own setup, see wetsuite's install guidelines.
!pip3 install -U --no-cache-dir --quiet https://github.com/knobs-dials/wetsuite-dev/archive/refs/heads/main.zip

# Purpose of this notebook



Seeing how well you can extract references to laws and jurisprudence from relatively free-form text.

Because there are few standards, and the ones there are are rarely used consistently.

You probably want to skim [notes__identifiers_and_references (notebook)](notes__identifiers_and_references.ipynb) first.

Optionally, take a glance at [references_lawnames](references_lawnames.ipynb) extracting the names of laws,
including less-official references in real-world use.

## What is our domain?

There will be references between documents like 
jurisprudence,
national law,
EU law, 
policy, 
and other official publications.

Extracting such references may be useful to varied tasks, such as 
- finding related documents by the references they share use, 
- estimate the generla topics a document covers,
- estimate what a document intends to do, by the legal authority it draws on.

<!-- -->

Particularly ECLI marks what it is in the identifier itself, so is unambiguous. 

The full form of a jci is similarly recognizable.


Many other things fall on a gliding scale.
LJN may just look like some letters and numbers.
The same goes for CELEX except there are some letters in a predictable yet otherwise-unusual place.
Such details metter if you want to avoid false positives.

Abbreviated references are more easily missed (false negatives), or lead to false positives.

Natural-language references may see a lot of _variation_ in how they are written down,
but their relative verbosity means that most cases are still fairly unambiguous.





### Some practical notes


TODO: MOVE WHAT IS RELEVANT TO NOTES_IDENTIFIERS


**Nerdnotes:**
* You could look for very specific text patterns, but things like regexps are likely to break under even minor variation
  * other classical formal grammars (CFGs and such) may also not quite be enough
  * something like [PEG](https://en.wikipedia.org/wiki/Parsing_expression_grammar)s are preferable

* There are some things that are essentially global aliases, shared through the community
  * e.g. referring to part of `Awb` is _very_ likely to be _Algemene wet bestuursrecht_ (BWBR0005537)
  
* Then there are what are essentially local aliases. Consider:
  - `"Ingevolge artikel 2, eerste lid, van de Wet op de adeldom (hierna: de Wet)"`
  - `"the European Convention on Human Right and Fundamental Freedoms (hereinafter 'the Convention')"`
  * which are important if you want to resolve _"article 6 of the Convention"_


**Practicalities**
* Some of the data we have is marked up, meaning there are links that are to a formal identifier, and have a less-formal name. 
  * for example, find `extref` tags in BWB and CVDR's XML form

* Some of the references are nicely in metadata fields. Knowing that can act as disambiguation of the field's value in a way we cannot easily do in free-flowing text.

* We also just have text that happens to be references - e.g. in free-flowing text.


* perhaps the most important is actually that it is often not clear where the reference ends. A simple example:

    TODO


**Practicalities we cannot cover easily, or at all:**
* References would be referring to the version valid at the time of reference, though that should be resolvable

* You may find a lot of **less-formal names for laws** from free-form text

* The same may not work as well for **policy** or **jurisprudence**. 
    * Say, "Rensing/Polak II" is practically unambiguous, even though it's **not** `ECLI:NL:HR:2005:AT4537`'s name
    * yet there are many links that will be named "Besluit", or another name that is unique only within that document and not something we should learn generally

* Not all references have the same function. Consider `"Commission Directive 2014/110/EU of 17 December 2014 amending Directive 2004/33/EC as regards ..."` or `"The substantive provisions of Directive 2004/33/EC were not affected by its amendment by Directive 2014/110/EU ..."` - it would not be hard to extract both references, but it is is up to you to notice that in terms of interpretation, one is referred to, and the other is only mentioned for completeness, or to decide there is a difference between mentions, supporing references, etc.



## Practical: informal references

In [14]:
import pprint, textwrap
from importlib import reload

import wetsuite.helpers.patterns
reload( wetsuite.helpers.patterns )

for test in [
    #'artikel 5:9, aanhef en onder b, Awb',
    #'artikel 8, aanhef en onder c, Wet bescherming persoonsgegevens (Wbp).',
    #'artikel 10, tweede lid, aanhef en onder e, van de Wob',
    #"artikel 4, tweede lid, aanhef en onder d, van het reglement van orde voor de ministerraad",
    #"Artikel 10, tweede lid, aanhef en onder e van de. Wet openbaarheid van bestuur",
    #"artikel 6:80 lid 1 aanhef en onder b BW",
    #'artikel 4, aanhef en onder d en g, van de standaardvoorwaarden',
    #'artikel 3.3, zevende lid, aanhef en onder i, Woo',
    #'artikel 15, aanhef en onder a of c (oud) RWN',

    '  artikel 81q lid 3c Gemeentewet  ',
    #'artikel 4:25, 4:35 van de Awb en artikel 10 van de ASV',
    #'Wabo, art. 2.12, eerste lid, aanhef en onder a, sub 1\xBA',
    #'artikel 79, aanhef en onder 6\xBA',
    #'artikel 3.3, zevende lid jo. vijfde lid, aanhef en onder i, Woo',
    #'artikel 142, eerste lid, aanhef en onder b (en derde lid), van het Wetboek van Strafvordering',
    'artikel 2, eerste, tweede, vijfde, en zesenzestigste lid',
    #'art. 166 lid 1 in verbinding met art. 353 lid 1 Rv',

    # '(Art. 81 RO)',
    # '(als bedoeld in art. 22 Rv.)',
    # 'artikel 166 Rv',
    # 'artikel 3:303 BW',
    # 'artikel 22 Rv',
    # 'artikel 4, tweede lid, aanhef en onder d, van het reglement van orde voor de ministerraad',
    # 'artikel 5:9, aanhef en onder b, Awb',
    # 'artikel 8, aanhef en onder c, Wet bescherming persoonsgegevens (Wbp)',
    # 'artikel 10, tweede lid, aanhef en onder e, van de Wob',
    # 'artikel 4, tweede lid, aanhef en onder d, van het reglement van orde voor de ministerraad',
    # 'Artikel 10, tweede lid, aanhef en onder e van de. Wet openbaarheid van bestuur',
    # 'artikel 6:80 lid 1 aanhef en onder b BW',
    # 'artikel 142, eerste lid, aanhef en onder b (en derde lid), van het Wetboek van Strafvordering',
    # 'artikel 4, aanhef en onder d en g, van de standaardvoorwaarden',
    # 'artikel 3.3, zevende lid, aanhef en onder i, Woo',
    # 'artikel 3.3, zevende lid jo. vijfde lid, aanhef en onder i, Woo',
    # 'artikel 79, aanhef en onder 6\xBA',
    # 'artikel 15, aanhef en onder a of c (oud) RWN',
    # 'Wabo, art. 2.12, eerste lid, aanhef en onder a, sub 1\xBA',
    # 'artikel 2, eerste, tweede, en vijfde lid, gestelde eisen voldoen. (Artikel 3, eerste lid, van de Tabaks- en rookwarenwet)',
    # 'artikel 2, eerste, tweede, vijfde, even zesenzestigste lid',


    #'LJN: AO7817, NJ 2005, 270',
    #'LJN: BO6106',
    #'LJN: ZC2793, NJ 1999, 685',
    #'LJN: AW2089, NJ 2006, 327',
]:
    #print()
    print()
    print( '=== %r ==='%test )
    matches = wetsuite.helpers.patterns.find_nonidentifier_references( test, debug=1 )
    for match in matches:
        #print( match )
        print('Matched: %r'%match['text'])
        #print('Details:')
        for k, v in match['details'].items():
            print( '    %20r:  %-r'%(k, v) )


=== '  artikel 81q lid 3c Gemeentewet  ' ===
SOFAR      [ARTIKEL 81Q] lid 3c Gemeentewet
MATCHED type=lid                 :   'lid 3c'                   1 chars after 
SOFAR      [ARTIKEL 81Q LID 3C] Gemeentewet
Matched: ''
               'artikel':  '81q'
                   'lid':  '3c'
               'lid_num':  []

=== 'artikel 2, eerste, tweede, vijfde, en zesenzestigste lid' ===
SOFAR      [ARTIKEL 2], eerste, tweede, vijfde, en zesenzestigste lid
MATCHED type=lid                 :   'eerste, tweede, vijfde, en zesenzestigste lid'  2 chars after 
SOFAR      [ARTIKEL 2, EERSTE, TWEEDE, VIJFDE, EN ZESENZESTIGSTE LID]
Matched: 'eerste, tweede, vijfde, en zesenzestigste'
               'artikel':  '2'
                   'lid':  'eerste, tweede, vijfde, en zesenzestigste'
               'lid_num':  [1, 2, 5, 66]


In [3]:
from wetsuite.helpers.patterns import reference_parse
reference_parse('artikel 5:9, aanhef en onder b, Awb')

MATCHED type=aanhefonder         :   'aanhef en onder b'        2 chars after 


[{'text': 'artikel 5:9, aanhef en onder b',
  'details': OrderedDict([('artikel', '5:9'),
               ('aanhefonder', 'aanhef en onder b')])}]

### Going further

It would be quite interesting that if 

- input like `"artikel 4, tweede lid, aanhef en onder d, van het reglement van orde voor de ministerraad"` would give any or all of:
   - BWBR0006501 
   - https://wetten.overheid.nl/jci1.3:c:BWBR0006501&artikel=4    or
     https://wetten.overheid.nl/jci1.3:c:BWBR0006501&paragraaf=2&artikel=4
   - jci1.3:c:BWBR0006501&paragraaf=2&artikel=4
   - ['4', 
      '2', 
      'Te dien einde beraadslaagt en besluit de raad onder meer over:', 
      'het bekendheid geven aan beleidsvoornemens in welke vorm dan ook, die van invloed kunnen zijn op de positie van het kabinet, '
      'of die belangrijke financiële consequenties kunnen hebben, benevens over beleidsvoornemens van een minister die het beleid van '
      'andere ministers kunnen raken en waarover het bereiken van overeenstemming niet mogelijk is gebleken;'
     ]

- **Internal references**, e.g. BWBR0004302's artikel 3, lid 1 has "artikel 2, eerste, tweede, en vijfde lid", which in that context ought to give
   - ['2', 
      ['1', '2', '5'], 
      ['Bij of krachtens algemene maatregel van bestuur worden in het belang van de volksgezondheid eisen gesteld aan tabaksproducten, ',
       'elektronische dampwaar, nicotinehoudende vloeistof en niet-nicotinehoudende vloeistof met betrekking tot maximumemissieniveaus ',
       'en ingrediënten en worden technische eisen gesteld, en kunnen methoden van onderzoek worden aangewezen die bij uitsluiting ',
       'beslissend zijn voor de vaststelling of met betrekking tot een product al dan niet aan de daaraan gestelde eisen is voldaan.,

              Bij of krachtens algemene maatregel van bestuur worden in het belang van de volksgezondheid eisen gesteld aan de verpakkingseenheid en de buitenverpakking van tabaksproducten en aanverwante producten. De eisen hebben betrekking op:

          a. de aanduidingen op verpakkingseenheden en buitenverpakkingen;

          b. de hoeveelheid of het aantal stuks dat is verpakt;

          c. de presentatie van het product;

          d. de sluiting, vorm, afmetingen en materiaal van de verpakkingseenheid of de buitenverpakking; en

          e. andere elementen van de verpakkingseenheid en de buitenverpakking die gebruikt kunnen worden om een onderscheid te maken tussen de verschillende merken van een tabaksproduct of een aanverwant product.

       'De verpakkingseenheid van elektronische dampwaar bevat een bijsluiter. Onze Minister stelt nadere regels over de inhoud van de bijsluiter.'
      ]
     ]




## Practical: Learning from LiDo


https://linkeddata.overheid.nl/front/portal/spiegel-lijstweergave?id=http%3A%2F%2Flinkeddata.overheid.nl%2Fterms%2Fjurisprudentie%2Fid%2FECLI%3ANL%3APHR%3A2011%3ABP5608&callback=&dates=&fields=&fq=%7B%21tag%3Dlink_richting%7Dlink_richting%3A%22inkomend%22&facet.field=%7B%21ex%3Dlink_richting%7Dlink_richting&facet.field=%7B%21ex%3Dobj_type%7Dobj_type&facet.field=%7B%21ex%3Dobj_organisatie_groep%7Dobj_organisatie_groep&facet.field=%7B%21ex%3Dobj_organisatie%7Dobj_organisatie&facet.field=%7B%21ex%3Dlink_type%7Dlink_type&facet.field=%7B%21ex%3Dobj_jaar%7Dobj_jaar



In [ ]:


import urllib.parse

urllib.parse.parse_qs('id=http%3A%2F%2Flinkeddata.overheid.nl%2Fterms%2Fjurisprudentie%2Fid%2FECLI%3ANL%3APHR%3A2011%3ABP5608&callback=&dates=&fields=&fq=%7B%21tag%3Dlink_richting%7Dlink_richting%3A%22inkomend%22&facet.field=%7B%21ex%3Dlink_richting%7Dlink_richting&facet.field=%7B%21ex%3Dobj_type%7Dobj_type&facet.field=%7B%21ex%3Dobj_organisatie_groep%7Dobj_organisatie_groep&facet.field=%7B%21ex%3Dobj_organisatie%7Dobj_organisatie&facet.field=%7B%21ex%3Dlink_type%7Dlink_type&facet.field=%7B%21ex%3Dobj_jaar%7Dobj_jaar')







